In [35]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string
import datetime

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [10]:
sql_sla_season = 'select * from sladb.dbo.tbl_sla_season'

In [16]:
sql_season_definition = 'select * from sladb.dbo.tbl_ref_sla_season_definition'

In [20]:
sql_season_date = 'select * from sladb.dbo.tbl_sla_season_date'

## Execute all of the sql queries and store the results

In [13]:
before_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [36]:
before_sla_season.head()

,season_id,season_desc,year_round,effective,effective_end,effective_start,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,"Year-round, not seasonal",True,False,2020-07-24,2014-01-01,2020-03-03 19:12:12.710,2020-07-28 20:16:56.850,2014-01-05,2020-07-25
1,2,"Beaches, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
2,3,"Ballfields, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
3,4,"NY Pause, COVID-19 Pandemic",True,False,2020-07-04,2020-03-22,2020-04-20 21:52:46.403,2020-07-01 15:20:36.193,2020-03-22,2020-07-04
4,6,New year round test season.,True,True,None,2020-07-24,2020-07-24 23:41:46.543,2020-07-28 18:31:55.867,2020-07-26,None


In [17]:
before_season_definition = pd.read_sql(con = engine, sql = sql_season_definition)

In [37]:
before_season_definition.head()

,season_date_ref_id,season_id,date_ref_fixed,month_name_desc,date_ref_day_number,day_name_desc,day_rank_id,date_type_id
0,1,1,True,January,1.0,None,None,1
1,2,1,True,December,31.0,None,None,2
2,3,2,True,May,1.0,None,None,1
3,4,2,True,October,1.0,None,None,2
4,5,3,True,March,1.0,None,None,1


In [21]:
before_season_date = pd.read_sql(con = engine, sql = sql_season_date)

In [38]:
before_season_date.head()

,season_date_id,season_id,effective_start,effective_end,date_category_id,effective_start_adj,effective_end_adj
0,1,1,2014-01-01,2020-07-24,1,2014-01-05,2020-07-25
1,2,2,2019-07-01,2019-10-01,1,2019-07-07,2019-10-05
2,4,2,2020-05-01,2020-07-25,1,2020-05-03,2020-07-25
3,6,3,2019-07-01,2019-10-31,1,2019-07-07,2019-11-02
4,8,3,2020-03-01,2020-07-25,1,2020-03-01,2020-07-25


In [29]:
timing = []

In [30]:
descriptions = []

In [31]:
results = []

## Before Tests

### First before test

In [32]:
timing.append('before')

In [33]:
descriptions.append('''Ensure that 'effective'==1 for every season that has 'effective_end_date'==null''')

In [34]:
results.append(
    len(before_sla_season[(pd.isnull(before_sla_season['effective_end']))
                          & (before_sla_season['effective'] == False)]) == 0)

### Second before test

In [95]:
timing.append('before')

In [96]:
descriptions.append('''Test that 'effective_end_adj' is after today if \
'effective'==1 AND 'effective_end' != null in tbl_sla_season''')

In [52]:
before_sla_season['effective_end_adj'] = pd.to_datetime(
    before_sla_season['effective_end_adj'])

In [59]:
today = datetime.datetime.today()

In [92]:
temp_df = before_sla_season[(before_sla_season['effective'] == True)
                  & (~pd.isnull(before_sla_season['effective_end']))].copy()

In [93]:
results.append(len(temp_df[temp_df['effective_end_adj'] > today])==len(temp_df))

### Third before test

In [105]:
timing.append('before')

In [106]:
descriptions.append('''Test that every season in tbl_sla_season has exactly two \
entries in tbl_ref_sla_season_definition''')

In [107]:
entry_count = []
for i in before_sla_season.season_id.values:
    entry_count.append(len(before_season_definition[before_season_definition['season_id']==i])==2)

In [108]:
results.append(False not in entry_count)

### Fourth before test

In [593]:
timing.append('before')

In [594]:
descriptions.append('''Test that the count of records in tbl_ref_sla_translation \
is equal to the count of sla_codes in tbl_ref_sla_code multiplied by the number \
of date_category_ids in tbl_ref_sla_season_category''')

In [595]:
results.append(
    len(before_sla_translation) == (before_sla_code['n_sla_codes'].values[0] *
                                    len(before_date_category)))

### Fifth before test

In [596]:
timing.append('before')

In [597]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [598]:
before_sla_translation_nodup = before_sla_translation.drop_duplicates()

In [599]:
results.append(len(before_sla_translation) == len(before_sla_translation_nodup))

### Sixth before test

In [600]:
timing.append('before')

In [601]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [602]:
before_unique_sla_translation = before_sla_translation.nunique()

In [603]:
results.append(
    ((before_unique_sla_translation['sla_id'] == before_count_sla_id)
     and (before_unique_sla_translation['date_category_id'] == len(
         before_date_category))
     and (before_unique_sla_translation['sla_code'] ==
          before_sla_code['n_sla_codes'].values[0])))

## Insert the sla_id values into tbl_ref_sla so that the after tests can be conducted

In [604]:
ninserts = 2

In [605]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [606]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [607]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [608]:
sla_inserts_df

,sla_id,sla_desc,sla_min_days,sla_max_days
0,G,Testing new SLA: G,1,2
1,H,Testing new SLA: H,1,2


In [609]:
# sla_inserts_df.to_sql('tbl_ref_sla', con = engine, index = False, if_exists = 'append')

## Execute all of the sql queries AFTER INSERT and store the results

In [610]:
after_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [611]:
after_count_sla_id

10

In [612]:
after_squared_count_sla_id = (after_count_sla_id)**2 

In [613]:
after_squared_count_sla_id

100

In [614]:
after_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [615]:
after_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,E
5,F
6,G
7,H
8,N
9,T


In [616]:
after_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [617]:
after_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,100,100


In [618]:
after_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [619]:
# after_date_category

In [620]:
after_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [621]:
# after_sla_translation

## After Tests

### First after test

In [622]:
timing.append('after')

In [623]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared \
is equal to the number of sla_codes in tbl_ref_sla_code''')

In [624]:
results.append(after_squared_count_sla_id == after_sla_code['n_sla_codes'].values[0])

In [625]:
results

[True, True, True, True, True, True, True]

### Second after test

In [626]:
timing.append('after')

In [627]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [628]:
results.append(after_sla_code['n_sla_codes'].values[0] == after_sla_code['max_sla_code'].values[0])

### Third after test

In [629]:
timing.append('after')

In [630]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [631]:
results.append(1 == after_sla_code['min_sla_code'].values[0])

### Fourth after test

In [632]:
timing.append('after')

In [633]:
descriptions.append(
    '''Test that the count of records in tbl_ref_sla_translation is \
    equal to the count of sla_codes in tbl_ref_sla_code multiplied by \
    the number of date_category_ids in tbl_ref_sla_season_category''')

In [634]:
results.append(len(after_sla_translation) == (after_sla_code['n_sla_codes'].values[0] * len(after_date_category)))

### Fifth before test

In [635]:
timing.append('after')

In [636]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [637]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [638]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Sixth after test

In [639]:
timing.append('after')

In [640]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [641]:
after_unique_sla_translation = after_sla_translation.nunique()

In [642]:
results.append(
    ((after_unique_sla_translation['sla_id'] == after_count_sla_id)
     and (after_unique_sla_translation['date_category_id'] == len(
         after_date_category)) and (after_unique_sla_translation['sla_code'] ==
                                    after_sla_code['n_sla_codes'].values[0])))

### Seventh after test

In [643]:
timing.append('after')

In [644]:
m = ninserts
n = before_count_sla_id

In [645]:
descriptions.append(
    '''Test that the number of rows insrted into tbl_ref_sla_code is equal to (m+n)**2-n**2 '''
)

In [646]:
results.append(after_sla_code.n_sla_codes.values[0] -
               before_sla_code.n_sla_codes.values[0] == (m + n)**2 - n**2)

### Eighth after test

In [647]:
timing.append('after')

In [648]:
descriptions.append(
    '''Test that the nmuber of rows in tbl_ref_sla_translation is equal to 2*(m+n)**2 '''
)

In [649]:
num_date_category_ids = len(after_date_category)

In [650]:
results.append(len(after_sla_translation) == num_date_category_ids * (m + n)**2)

### Ninth after test

In [651]:
timing.append('after')

In [652]:
descriptions.append('''After removing the original rows, \
test the uniqueness of the rows in tbl_ref_sla_translation''')

In [653]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [654]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Tenth after test

In [655]:
timing.append('after')

In [656]:
descriptions.append(
    '''Ensuring the correct number of rows was inserted into tbl_ref_sla_trnalsation'''
)

In [657]:
added_rows_df = pd.merge(after_sla_translation,
         before_sla_translation,
         on=['sla_id', 'sla_code', 'date_category_id'],
         indicator=True,
         how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [658]:
results.append(len(added_rows_df)==(len(after_sla_translation) - len(before_sla_translation)))

## List of tuples with final test results

In [659]:
list(zip(timing, descriptions, results))

[('before',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of sla_codes in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the minimum sla_code in tbl_ref_sla_code is equal to 1',
  True),
 ('before',
  'Test that the count of records in tbl_ref_sla_translation is equal to the count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids in tbl_ref_sla_season_category',
  True),
 ('before',
  'Test the uniqueness of the rows in tbl_ref_sla_translation',
  True),
 ('before',
  'Test that all sla_ids, sla_codes and date_category_ids have been     accounted for in tbl_ref_sla_translation',
  True),
 ('after',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of sla_codes in tbl_ref_sla_code',
  True),
 ('after',
  'Test that the number of sla_codes in tbl_ref

In [109]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,test_desc,before/after,result
0,before,Ensure that 'effective'==1 for every season th...,True
1,before,Test that 'effective_end_adj' is after today i...,True
2,before,Test that every season in tbl_sla_season has e...,True
